In [2]:
import pandas as pd # Pandas is an open source library providing high-performance, easy-to-use data structures and data analysis tools for Python
from functools import reduce
import numpy as np
from math import isnan
import operator
import datetime

ga_cleaned_data_folder = "C:/Users/huixin/Dropbox/research/menoplan/beta_phase/ga_data/cleaned_data/"
menoplan_dir = "C:/Users/huixin/Dropbox/research/menoplan/beta_phase/"
ga_data_folder = "C:/Users/huixin/Dropbox/research/menoplan/beta_phase/ga_data/"

This notebook we merge qualtrics data provided by Prof Leslie Synder with: 
    1. Menopause Results data
    2. Decision filters data
    3. Decision tool /symptoms data
    4. OS type
    5. Page View Duration 
    6. Page View Count
    7. Num of Sessions

In [38]:
qualtrics_data = pd.read_csv("C:/Users/huixin/Dropbox/research/menoplan/beta_phase/qualtrics_data/qualtrics_02nov.csv")
print(len(qualtrics_data.columns), len(qualtrics_data))
#menopause_results.iloc[:, 1:].sum().sort_values()#.sum() #161 users
menopause_results = pd.read_csv("C:/Users/huixin/Dropbox/research/menoplan/beta_phase/ga_data/cleaned_data/nov02_am_i_in_menopause.csv")
menopause_results['sumResult'] = np.count_nonzero(menopause_results.iloc[:,1:], axis = 1)
decision_filters = pd.read_csv("C:/Users/huixin/Dropbox/research/menoplan/beta_phase/ga_data/cleaned_data/nov02_decision_filter_clicks.csv")
decision_tools = pd.read_csv("C:/Users/huixin/Dropbox/research/menoplan/beta_phase/ga_data/cleaned_data/nov02_decision_tool_clicks.csv")
decision_filters.iloc[:, 1:]#.sum().sort_values() #181 rows
decision_filters = pd.merge(decision_tools[[ 'IPAddress', 'supplement']], decision_filters, on= "IPAddress")
decision_filters['selectFilter'] = np.count_nonzero(decision_filters.iloc[:,1:], axis = 1)
decision_filters['clickFilter']= decision_filters.iloc[:, 1:].sum(axis = 1)
decision_filters = decision_filters.sort_values(by='selectFilter', ascending=False)
decision_tools = decision_tools.drop(columns= 'supplement')

226 479


In [69]:
qualtrics_data['Q_RecaptchaScore0'].replace(' ', np.nan, regex=True)
print((pd.to_numeric(qualtrics_data['Q_RecaptchaScore0'],errors = 'coerce').value_counts()))
print(237/479*100)


1.0    164
0.9     75
0.8      1
3.0      1
0.4      1
Name: Q_RecaptchaScore0, dtype: int64
49.478079331941544


In [98]:
#merge qualtrics data with respective GA data
qualtrics_menopause_res = pd.merge(qualtrics_data,menopause_results, on="IPAddress", how='left')
qualtrics_decision_tool_res = pd.merge(qualtrics_data,decision_tools, on="IPAddress", how='left')
qualtrics_decision_filter_res = pd.merge(qualtrics_data,decision_filters, on="IPAddress", how='left')

decision_tools['selectSym'] = np.count_nonzero(decision_tools.iloc[:,1:], axis = 1)
decision_tools.sort_values(by='selectSym', ascending=False)
decision_tools['clickSym']= decision_tools.iloc[:, 1:].sum(axis = 1)

In [116]:
# compile the list of dataframes you want to merge
data_frames = [qualtrics_decision_tool_res,qualtrics_decision_filter_res, qualtrics_menopause_res]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['IPAddress'], how='inner'), data_frames)
df_merged.drop(df_merged.filter(regex=r'(_y$|_x$)').columns.tolist(),axis=1, inplace=True)
#this df contains  page views, n sessions and session durations from a csv file
#each IP has multiple rows because there are several unique hours/occassions of the day each IP visited a Page
df = pd.read_csv("C:/Users/huixin/Dropbox/research/menoplan/beta_phase/ga_data/02nov_pgview_nsess_sessdur.csv")
df['sumVisits'] = df.groupby(['IP Address'])['Hour of Day'].transform('count')
df = df.sort_values(by=['sumVisits', 'IP Address', 'Hour of Day' , 'Session Duration'], ascending =False)
df =df.drop(columns="Sessions")

In [100]:
df.to_csv(ga_cleaned_data_folder + "date_of_each_visit_by_IP.csv")

In [101]:
menoplan_tool_by_date = pd.read_csv(ga_data_folder + '02nov_menoplan_tool_clicks.csv')
#group by IPAddress and get the Hour of Day and count sum of clicks during all Hour of Days
menoplan_tool_by_date['sumEventClicks'] = menoplan_tool_by_date.groupby(['IP Address'])['Hour of Day'].transform('count')
menoplan_tool_by_date = menoplan_tool_by_date.sort_values(by=[ 'sumEventClicks', 'IP Address', 'Hour of Day'], ascending=False)
#this df has multiple rows for a single user - each row is the date of the clicks
menoplan_tool_by_date.to_csv(ga_cleaned_data_folder + "nov02_date_of_each_click_by_IP.csv")
#group by IP, transform  Event Label by applying lambda function by joining the strings with ','
menoplan_tool_by_date['allEvents'] = menoplan_tool_by_date[['IP Address', 'Event Label']].groupby(['IP Address',])['Event Label'].transform(lambda x: ','.join(x))
menoplan_tool_by_date[['IP Address','allEvents']].drop_duplicates() #drop rows if allViewedPages, ResponseId and EndDate are the same
menoplan_tool_by_date.reset_index(drop=True, inplace=True) #reset index
#get mean of total events, and mean of sessions across rows after grouping by IP and the list of all events 
eventsSummary = menoplan_tool_by_date.groupby(['IP Address','allEvents'], as_index=False).agg({'Total Events':'mean', 'Sessions':"mean"})# 'Pageviews': 'mean'})
#eventsSummary = eventsSummary.drop(columns=[ 'Sessions', "Pageviews"])

In [102]:
#get page duration data from csv
pageDur = pd.read_csv(menoplan_dir + 'merged_analysis/merged_pageDuration_02nov.csv').iloc[:, 0:81]
new_names = [(i,i+'Dur') for i in pageDur.iloc[:, 2:81].columns.values] #get all cols related to pages
pageDur.rename(columns = dict(new_names), inplace=True) #add suffix Dur for Duration
pageDur = pageDur.iloc[:, 1:81]  
pageDur['sumPageDur'] = pageDur.iloc[:,1:81].sum(axis=1) 

,IPAddress,?s=AnxietyDur,?s=TransgenderDur,?s=am+i+in+menopauseDur,Day IndexDur,aboutDur,acupunctureDur,am-i-in-menopauseDur,avoiding-triggersDur,avoiding-weight-gainDur,...,vitaminsDur,weight-gainDur,what-if-i-have-breast-cancerDur,what-if-im-transgender-or-nonbinaryDur,who-we-areDur,womens-storiesDur,yogaDur,insomnia-severity-index-resultsDur,anxiety-assessment-resultsDur,sumPageDur
0,100.1.197.247,NaN,NaN,NaN,NaN,NaN,NaN,126.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,408.0
1,100.1.210.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,633.0
2,104.4.139.57,NaN,NaN,NaN,NaN,NaN,NaN,187.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,506.0
3,104.7.29.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,48.0
4,107.10.231.221,NaN,NaN,NaN,NaN,NaN,NaN,32.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,64.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,99.185.57.102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.0,...,NaN,107.0,NaN,NaN,NaN,321.0,NaN,0.0,0.0,918.0
222,99.196.242.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,482.0
223,99.32.58.89,NaN,NaN,NaN,NaN,15.0,NaN,54.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,320.0
224,99.44.25.230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,573.0


In [103]:
#get page view count data from csv
pageCounts = pd.read_csv(menoplan_dir + 'merged_analysis/merged_pageCount_02nov.csv').iloc[:, 1:81]
new_names = [(i,i+'Count') for i in pageCounts.iloc[:, 1:81].columns.values]
pageCounts.rename(columns = dict(new_names), inplace=True)
pageCounts['sumPageCounts'] = pageCounts.iloc[:,1:81].sum(axis=1)

,IPAddress,?s=AnxietyCount,?s=TransgenderCount,?s=am+i+in+menopauseCount,Day IndexCount,aboutCount,acupunctureCount,am-i-in-menopauseCount,avoiding-triggersCount,avoiding-weight-gainCount,...,vitaminsCount,weight-gainCount,what-if-i-have-breast-cancerCount,what-if-im-transgender-or-nonbinaryCount,who-we-areCount,womens-storiesCount,yogaCount,insomnia-severity-index-resultsCount,anxiety-assessment-resultsCount,sumPageCounts
0,100.1.197.247,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
1,100.1.210.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
2,104.4.139.57,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,10.0
3,104.7.29.81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,107.10.231.221,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,99.185.57.102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,16.0
222,99.196.242.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0
223,99.32.58.89,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
224,99.44.25.230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0


In [104]:
os = pd.read_csv(ga_cleaned_data_folder + "02nov_os.csv")
os.rename(columns={ "IP Address": "IPAddress"})

,IPAddress,Operating System
0,1.127.30.239,iOS
1,100.0.48.40,iOS
2,100.1.116.150,iOS
3,100.1.197.247,Android
4,100.1.210.47,iOS
...,...,...
641,99.8.85.105,Windows
642,99.82.254.180,Android
643,99.89.44.176,Android
644,99.92.172.96,iOS


In [105]:
eventsSummary = eventsSummary.rename(columns={ "IP Address": "IPAddress"})
df_merged = pd.merge(df_merged, eventsSummary, on='IPAddress', how='left') #merge with events data
df_merged = pd.merge(df_merged, os, on='IPAddress', how='left')#merge with OS

In [113]:
#pageCounts.loc[pageCounts['IPAddress'] == '99.82.254.180' ] #for validating with Google Analytics original data
col = list(df_merged.columns)
filter_col = [col for col in df_merged if col.startswith('sum')]
filter_col

['sumResult', 'sumPageDur', 'sumPageCounts']

df_merged has all page counts, event clicks, os info, page view duration

In [108]:
df_merged = pd.merge(df_merged, pageCounts, on='IPAddress', how='left')#merge with page counts

,IPAddress,common-depression.png,common-hot-flashes.png,common-insomnia.png,dry-eyes2.png,other-aches-pains.png,other-bladder.png,other-brain-fog.png,other-dryness.png,other-weight-gain.png,...,vitaminsCount,weight-gainCount,what-if-i-have-breast-cancerCount,what-if-im-transgender-or-nonbinaryCount,who-we-areCount,womens-storiesCount,yogaCount,insomnia-severity-index-resultsCount,anxiety-assessment-resultsCount,sumPageCounts
0,100.1.197.247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
1,100.1.210.47,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
2,100.15.208.186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100.16.5.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,104.4.139.57,0.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,99.23.156.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475,99.32.58.89,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
476,99.44.25.230,0.0,3.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0
477,99.62.239.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
df_merged.to_csv(menoplan_dir + 'merged_analysis/02nov_merged_pageviews_events_os.csv')

In [156]:
import datetime
df_merged['startdate0'] = pd.to_datetime(df_merged['startdate0'], errors='coerce')
df_merged['EndDate'] = pd.to_datetime(df_merged['EndDate'], errors='coerce')
df_merged['surveyDur']=(df_merged['EndDate'] - df_merged['startdate0'])
(df_merged['surveyDur']).dropna().sort_values().tail(10)

336    0 days 02:57:47
459    0 days 03:31:30
309    0 days 06:37:21
107    0 days 07:53:47
348    0 days 21:06:16
420    2 days 20:56:50
114    3 days 03:32:20
306   18 days 10:47:11
440   46 days 06:15:29
320   47 days 07:45:55
Name: surveyDur, dtype: timedelta64[ns]

Merge OS data with feedback from qualtrics

In [ ]:
os_fdbck.to_csv(menoplan_dir + 'merged_analysis/os_feedback.csv')
fdbk = pd.read_excel(menoplan_dir + "/qualtrics_data/qualtrics_open_ended.xlsx", engine='openpyxl')
fdbk.columns
os_fdbck = pd.merge(os,fdbk , on = "IPAddress" ).sort_values(by="Operating System")
os_fdbck[os_fdbck['Operating System'] == 'iOS']
os_fdbck.to_csv(menoplan_dir + 'merged_analysis/os_feedback.csv')
